In [ ]:
!pip install pyod

# Step 1: Outlier Treatment

## 1. Outlier Detection using pyOD

In [1051]:
import pandas as pd
import numpy as np

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF

from sklearn.preprocessing import MinMaxScaler



In [1052]:
df=pd.read_csv("null from alpha vantage/final_result_after_kmeans.csv")
df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)

In [1053]:
random_state=3
outliers_fraction = 0.05


In [ ]:
Angle-based Outlier Detector (ABOD)
Cluster-based Local Outlier Factor (CBLOF)
Histogram-base Outlier Detection (HBOS)

In [1054]:
classifiers = {
        'ABOD': ABOD(contamination=outliers_fraction),
        'CBLOF':CBLOF(contamination=outliers_fraction,check_estimator=False, random_state=random_state),
        'Feature Bagging':FeatureBagging(LOF(n_neighbors=35),contamination=outliers_fraction,check_estimator=False,random_state=random_state),
        'HBOS': HBOS(contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,random_state=random_state),
        'KNN': KNN(contamination=outliers_fraction),
        'Average KNN': KNN(method='mean',contamination=outliers_fraction)
}

In [29]:
outliers_fraction = 0.1

num_cols=df.select_dtypes(include=['float64','int64']).columns
num_cols=num_cols[0:177]
df_num=df[num_cols]
cols=df_num.columns

scaler = MinMaxScaler(feature_range=(0, 1))
df_num = scaler.fit_transform(df_num)

df_num=pd.DataFrame(df_num)
df_num.columns=cols

for i, (clf_name, clf) in enumerate(classifiers.items()):
   

    clf.fit(df_num)

    y_train_scores = clf.decision_scores_ 
    y_train_pred = clf.labels_

    df['Outliers'+' ' +clf_name]=y_train_pred


  




/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


In [33]:
df['sum_outlier']=df['Outliers ABOD']+df['Outliers CBLOF']+df['Outliers Feature Bagging']+df['Outliers HBOS']+df['Outliers Isolation Forest']+df['Outliers KNN']+df['Outliers Average KNN']


    

In [71]:
df_outlier_removed=df[df['sum_outlier']<5]

In [72]:
df_outlier_removed.drop(['Outliers ABOD','Outliers CBLOF','Outliers Feature Bagging','Outliers HBOS'
                        , 'Outliers Isolation Forest','Outliers KNN','Outliers Average KNN','sum_outlier' ],axis=1,inplace=True)

In [74]:
df_outlier_removed.to_csv('null from alpha vantage/final result after outlier removal')

## 2. Winsorization (Replacing the outlier by upper or lower cap)


In [75]:
df= pd.read_csv("null from alpha vantage/final result after outlier removal")
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [79]:
num_cols=df.select_dtypes(include=['float64','int64']).columns
num_cols=num_cols[0:177]

In [80]:
for cols in num_cols:
    q1=df[cols].quantile(0.1)
    q2=df[cols].quantile(0.5)
    q3=df[cols].quantile(0.9)
    iqr=q3-q1
    ul=q3+1.5*iqr
    ll=q1-1.5*iqr
    df[cols]=df[cols].apply(lambda x:ul if x>ul else x )
    df[cols]=df[cols].apply(lambda x:ll if x<ll else x )

In [81]:
count=0
for cols in num_cols:
    q1=df[cols].quantile(0.1)
    q2=df[cols].quantile(0.5)
    q3=df[cols].quantile(0.9)
    iqr=q3-q1
    ul=q3+1.5*iqr
    ll=q1-1.5*iqr
    print(cols)
    print(df[df[cols]>ul][cols].count())
    print(df[df[cols]<ll][cols].count())
    print()
    count=count+1

Revenue
0
0

Revenue Growth
0
0

Cost of Revenue
0
0

Gross Profit
0
0

R&D Expenses
0
0

SG&A Expense
0
0

Operating Expenses
0
0

Operating Income
0
0

Interest Expense
0
0

Earnings before Tax
0
0

Income Tax Expense
0
0

Net Income - Non-Controlling int
0
0

Net Income - Discontinued ops
0
0

Net Income
0
0

Preferred Dividends
0
0

Net Income Com
0
0

EPS
0
0

EPS Diluted
0
0

Weighted Average Shs Out
0
0

Weighted Average Shs Out (Dil)
0
0

Dividend per Share
0
0

Gross Margin
0
0

EBITDA Margin
0
0

EBIT Margin
0
0

Profit Margin
0
0

Free Cash Flow margin
0
0

EBITDA
0
0

EBIT
0
0

Consolidated Income
0
0

Earnings Before Tax Margin
0
0

Cash and cash equivalents
0
0

Short-term investments
0
0

Cash and short-term investments
0
0

Receivables
0
0

Inventories
0
0

Total current assets
0
0

Property, Plant & Equipment Net
0
0

Goodwill and Intangible Assets
0
0

Long-term investments
0
0

Tax assets
0
0

Total non-current assets
0
0

Total assets
0
0

Payables
0
0

Short-term d

In [82]:
df.to_csv('null from alpha vantage/final result after outlier capping')

## Step 2 Stastical Analysis (Pearson correlation) - to identify & drop similar columns

In [160]:
import scipy.stats as stats
import pandas as pd
from scipy.stats import pearsonr

In [161]:
df= pd.read_csv("null from alpha vantage/final result after outlier capping")

In [162]:
df.drop(['Unnamed: 0','operatingProfitMargin','Net Income - Discontinued ops'],axis=1,inplace=True)

In [163]:
num_cols=df.select_dtypes(include=['float64','int64']).columns

In [164]:
# dropping year, class, next year price var
num_cols=num_cols[0:176]
df_num=df[num_cols]

In [165]:
from sklearn.preprocessing import StandardScaler
df_num=df[num_cols]
ss=StandardScaler()
cols=df_num.columns
df_num=ss.fit_transform(df_num)

df_num=pd.DataFrame(df_num,columns=cols)

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#df_num=df[num_cols]
#cols=df_num.columns
#scaler = MinMaxScaler(feature_range=(0, 1))
#df_num = scaler.fit_transform(df_num)
#df_num=pd.DataFrame(df_num,columns=cols)

In [166]:
df_stats=pd.DataFrame()
for cols_main in df_num.columns:
    for cols_sub in df_num.columns:
            df_stats.loc[cols_main,cols_sub]= pearsonr(df_num[cols_main],df_num[cols_sub])[0]
            
    
    
   

In [167]:
df_corr=pd.DataFrame()
for cols in df_stats.columns:
    l1=list(df_stats[df_stats[cols]>0.9].index)
    l1.remove(cols)
    count=0
    for item in l1:
        df_corr.loc[cols,'values'+str(count)]=l1[count]
        count=count+1

    
   

In [176]:

col_removed=['Consolidated Income', 'Net Income Com','EBIT', 'Earnings before Tax', 'Operating Income','EPS Diluted',
'Weighted Average Shs Out (Dil)', 'Profit Margin', 'pretaxProfitMargin', 'netProfitMargin', 'Earnings Before Tax Margin', 
'Average Receivables', 'Average Inventory', 'Total liabilities', 'Average Payables', 'Long-term debt', 'PFCF ratio',
'POCF ratio', 'Return on Tangible Assets', 'ROIC', 'longtermDebtToCapitalization', 'cashPerShare', 'Invested Capital',
'Net Income Growth','EPS Diluted Growth']

In [177]:
df.drop(col_removed,axis=1,inplace=True)

In [187]:
df.to_csv('null from alpha vantage/final result after removing statstical similar columns.csv')

In [276]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 20)
#pd.set_option('display.max_colwidth', -1)

In [188]:
#Heat map

In [ ]:
#import pandas as pd
#import numpy as np
#import seaborn as sns

#X = df_num 

#corrmat = df_num.corr()
#top_corr_features = corrmat.index
#plt.figure(figsize=(900,900))
#plot heat map
#g=sns.heatmap(df_num[top_corr_features].corr(),annot=True,cmap="RdYlGn")

# Step 3 - Feature selection

# A  Univariate feature selection

## 1 f test classification

In [230]:
df=pd.read_csv('null from alpha vantage/final result after removing statstical similar columns.csv')

In [231]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [232]:
num_cols=df.select_dtypes(include=['float64','int64']).columns
num_cols=num_cols[0:149]
df_num=df[num_cols]

In [233]:
from sklearn.preprocessing import StandardScaler
df_num=df[num_cols]
ss=StandardScaler()
cols=df_num.columns
df_num=ss.fit_transform(df_num)

df_num=pd.DataFrame(df_num,columns=cols)

In [234]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
sel_f = SelectKBest(f_classif, k=90)
df_f = sel_f.fit_transform(df_num, df['Class'])
print(sel_f.get_support())

[ True False  True  True False  True  True False  True False  True False
  True  True  True  True  True  True  True  True  True False  True  True
 False  True False  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True False False False  True False  True  True False  True False
  True False  True  True False  True False False False  True  True False
 False False  True  True False  True False  True  True False  True False
  True  True False  True  True  True False False  True False False  True
  True  True  True False  True  True  True  True  True False False False
  True  True False  True  True  True  True False False False False  True
  True  True False  True  True False  True False False False False  True
 False False False False False  True  True  True False False False False
  True False False  True  True]


In [235]:
df_univar=pd.DataFrame(sel_f.get_support(),index=df_num.columns)

In [236]:
df_univar.loc[df_univar[0] == False]

,0
Revenue Growth,False
R&D Expenses,False
Interest Expense,False
Net Income - Non-Controlling int,False
Preferred Dividends,False
...,...
Free Cash Flow growth,False
Receivables growth,False
Inventory Growth,False
Book Value per Share Growth,False


In [237]:
df_univar.rename(columns={0:'f test'},inplace=True)

## 2 use mutual_info_classif test

In [238]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
sel_mutual = SelectKBest(mutual_info_classif, k=90)
df_mutual = sel_mutual.fit_transform(df_num, df['Class'])
print(sel_mutual.get_support())

[ True  True False  True  True False False  True False False  True False
  True  True False False  True  True  True  True  True False  True  True
  True  True  True False  True False  True  True  True False  True  True
 False False  True False False  True  True  True  True  True  True  True
 False  True  True False  True  True False  True  True False False  True
  True False  True  True  True  True False  True  True  True  True  True
 False  True False False False  True False False  True  True  True  True
  True  True  True  True  True  True False False False False False  True
 False  True  True False False  True  True  True  True  True False False
 False  True  True  True  True  True  True False  True  True False  True
 False False  True  True  True False False False False False False  True
 False  True False  True False  True  True  True  True False False False
 False  True False  True False]


In [239]:
df_mi=pd.DataFrame(sel_mutual.get_support(),index=df_num.columns)

In [240]:
df_mi.loc[df_mi[0] == False]

,0
Cost of Revenue,False
SG&A Expense,False
Operating Expenses,False
Income Tax Expense,False
Net Income - Non-Controlling int,False
...,...
Receivables growth,False
Inventory Growth,False
Asset Growth,False
Debt Growth,False


In [241]:
df_mi.rename(columns={0:'MI'},inplace=True)

# B Recursive feature elimination

## 1  use the logistic regression as the model

In [242]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
model_logistic = LogisticRegression(solver='lbfgs', max_iter=1000)
sel_rfe_logistic = RFE(estimator=model_logistic, n_features_to_select=90, step=1)
df_rfe_logistic = sel_rfe_logistic.fit_transform(df_num, df['Class'])
print(sel_rfe_logistic.get_support())

[ True  True  True  True  True  True False False False False  True  True
  True False False  True  True  True  True  True  True  True False  True
  True  True False False  True False  True  True  True False False  True
  True  True  True False False  True False False  True  True False  True
  True  True False False  True  True False  True  True  True  True False
  True  True False  True False False False False  True False False False
 False  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True False  True False False
  True False False False  True  True  True False False  True False  True
  True False False  True False False  True False  True False  True  True
 False  True  True  True  True  True  True  True False  True  True  True
  True False  True  True  True  True False  True False False False  True
 False False False False False]


In [243]:
df_lr=pd.DataFrame(sel_rfe_logistic.get_support(),index=df_num.columns)

In [244]:
df_lr.loc[df_lr[0] == False]

,0
Operating Expenses,False
Interest Expense,False
Income Tax Expense,False
Net Income - Non-Controlling int,False
Weighted Average Shs Out,False
...,...
Asset Growth,False
Book Value per Share Growth,False
Debt Growth,False
R&D Expense Growth,False


In [245]:
df_lr.rename(columns={0:'lr'},inplace=True)

## 2 use the random forest as the model

In [246]:
from sklearn.ensemble import RandomForestClassifier
model_tree = RandomForestClassifier(random_state=100, n_estimators=50)
sel_rfe_tree = RFE(estimator=model_tree, n_features_to_select=90, step=1)
df_rfe_tree = sel_rfe_tree.fit_transform(df_num, df['Class'])
print(sel_rfe_tree.get_support())


[False  True False False False  True  True False  True False  True False
  True  True False  True False  True False  True  True False False False
 False False False False False False False  True False False  True False
 False False False False  True  True False False False  True  True False
 False  True False False False  True  True  True False  True  True  True
 False  True False  True  True  True  True False  True  True  True  True
  True  True  True  True False  True False  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True False  True
  True False  True  True  True  True  True  True  True False False  True
  True False  True False False  True False False False  True  True False
 False  True  True  True  True  True False False  True  True False  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True False  True]


In [247]:
df_rf=pd.DataFrame(sel_rfe_tree.get_support(),index=df_num.columns)

In [248]:
df_rf.loc[df_rf[0] ==False]

,0
Revenue,False
Cost of Revenue,False
Gross Profit,False
R&D Expenses,False
Interest Expense,False
...,...
Days Payables Outstanding,False
Days of Inventory on Hand,False
Inventory Turnover,False
Dividends per Share Growth,False


In [249]:
df_rf.rename(columns={0:'rf'},inplace=True)

In [250]:
df_feature=pd.concat([df_univar,df_mi,df_lr,df_rf],axis=1)

In [251]:
df_feature=df_feature.astype(int)

In [252]:
df_feature['total']=df_feature['f test']+df_feature['MI']+df_feature['lr']+df_feature['rf']

In [271]:
df_feature[(df_feature['total']<2)  ]

,f test,MI,lr,rf,total
Interest Expense,0,1,0,0,1
Net Income - Non-Controlling int,0,0,0,0,0
Preferred Dividends,0,0,1,0,1
Dividend per Share,1,0,0,0,1
Short-term investments,0,0,1,0,1
...,...,...,...,...,...
Receivables Turnover,0,0,0,1,1
Inventory Turnover,0,0,1,0,1
Free Cash Flow growth,0,0,0,1,1
Receivables growth,0,0,0,1,1


In [682]:
df_feature.loc['R&D Expense Growth',:]

f test    1
MI        1
lr        0
rf        0
total     2
Name: R&D Expense Growth, dtype: int64

In [263]:
col_removed=['Net Income - Non-Controlling int', 'Tax assets', 'Intangibles to Total Assets','Receivables growth'
                        ,'operatingCycle','ebtperEBIT','Acquisitions and disposals','Issuance (repayment) of debt']

In [264]:
df.drop(col_removed,axis=1,inplace=True)

In [283]:
df.drop(['Preferred Dividends','Short-term investments'],axis=1,inplace=True)

In [284]:
df.to_csv('null from alpha vantage/final result after removing by feature selection.csv')

# Step 4 Data Transformation

## 1 Binning the numerical features

In [1023]:
df=pd.read_csv("null from alpha vantage/final result after removing by feature selection.csv")

In [1024]:
cols_bin=['Revenue Growth','Gross Profit Growth','SG&A Expenses Growth',
'Operating Income Growth','EBIT Growth','EPS Growth','Weighted Average Shares Growth',
'Weighted Average Shares Diluted Growth',   'Asset Growth',
'Debt Growth', 'Book Value per Share Growth', 'Operating Cash Flow growth','Free Cash Flow growth',
'R&D Expense Growth','Inventory Growth']



In [ ]:
#these columns have not yet been bined, But can be consider for binning in future
cols_not_yet_binned=['ROE',
'returnOnCapitalEmployed','returnOnAssets','Free Cash Flow Yield','Earnings Yield',
'EPS','Gross Margin','EBIT Margin','Free Cash Flow margin','Current ratio','quickRatio','cashRatio',

'Revenue per Share','Net Income per Share','Cash per Share','Operating Cash Flow per Share','Capex per Share',
'Shareholders Equity per Share','Interest Debt per Share',
'Book Value per Share','Tangible Book Value per Share',
'Free Cash Flow per Share',

'Days Payables Outstanding','Days Sales Outstanding','Dividend Yield',

'PE ratio','priceToFreeCashFlowsRatio','Interest Coverage','Days of Inventory on Hand','Inventory Turnover',
'Payables Turnover','priceEarningsToGrowthRatio',

'Receivables Turnover','fixedAssetTurnover','assetTurnover',
'cashFlowCoverageRatios','shortTermCoverageRatios',
'capitalExpenditureCoverageRatios','dividendpaidAndCapexCoverageRatios',
'enterpriseValueMultiple','Dividend per Share',

'Income Quality','cashConversionCycle',
'Dividends per Share Growth','R&D to Revenue',
'Price to Sales Ratio','SG&A to Revenue','nIperEBT','Payout Ratio','totalDebtToCapitalization','Net Debt to EBITDA',
'Debt to Equity','Debt to Assets','PTB ratio','Stock-based compensation to Revenue',
'operatingCashFlowSalesRatio','priceToOperatingCashFlowsRatio',
'Capex to Operating Cash Flow','Capex to Revenue','Capex to Depreciation','freeCashFlowOperatingCashFlowRatio',
'EV to Operating cash flow','EV to Free cash flow',
'EV to Sales','companyEquityMultiplier']

In [1025]:
cols_left=[]
for cols in cols_bin:
    bin_labels_5 = [0,1, 2,3,4]
    try:
        df[cols]= pd.cut(df[cols], 5,labels=bin_labels_5)
    except:
        print(cols)

In [1026]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [1027]:
df.to_csv('null from alpha vantage/final result after binning.csv')

## 2 Encoding the categorical variable 

In [1028]:
df=pd.read_csv('null from alpha vantage/final result after binning.csv')

In [1029]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [1030]:
df=pd.get_dummies(df,columns=['Sector'])

# 3 Scaling the data

In [1032]:
num_cols=df.select_dtypes(include=['float64','int64']).columns
num_cols=num_cols[0:139]


In [1033]:
num_cols1=[]
for cols in num_cols:
    if cols not in cols_bin:
        num_cols1.append(cols)
        

In [1035]:
df_num=df[num_cols1]

In [1036]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
cols=df_num.columns
df_num=ss.fit_transform(df_num)

df_num=pd.DataFrame(df_num,columns=cols)

In [1037]:
rem_cols=[]
for cols in df.columns:
    if (cols not in df_num.columns):
        rem_cols.append(cols)
        

In [1041]:
df_concat=pd.concat((df_num,df[rem_cols]),join='outer',axis=1)

In [1049]:
df_concat.to_csv('null from alpha vantage/final result after scaling-encoding.csv')

# END